In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import torch
import os
import sys
sys.path.append("../")

In [2]:
atlas_data_dir = "/data/cb/scratch/datasets/atlas"
atlas_dynamicslabels_dir = "/data/cb/scratch/datasets/atlas_dynamics_labels"
config_dir = "../trained/configs"

In [3]:
import yaml
from pathlib import Path

# Load data config
with open(config_dir+"/data/atlas_config.yaml", "r") as file:
    dataconfig = yaml.safe_load(file)
    
# Load train config
with open(config_dir+"/model/dynaprot_simple.yaml", "r") as file:
    modelconfig = yaml.safe_load(file)
    
modelconfig["data_config"] = dataconfig
    
print(modelconfig)

from dynaprot.data.datasets import DynaProtDataset, OpenFoldBatchCollator

dataset = DynaProtDataset(dataconfig)


batch_prots = next(iter(dataloader))

for k in batch_prots.keys():
    # print(k,batch_prots[k])
    print(f"{k}: {batch_prots[k].shape}")

{'model_params': {'num_ipa_blocks': 8, 'd_model': 128}, 'train_params': {'batch_size': 32, 'epochs': 50, 'learning_rate': 0.001}, 'eval_params': {'loss_weights': {'resi_gaussians': {'mse_means': 1.0, 'kldiv': 1.0}, 'resi_rmsf': 'None', 'resi_rmsd': 'None', 'resi_rg': 'None'}}, 'model_checkpoints': '${repo_dir}/trained/checkpoints', 'logs': '/path/to/logs', 'results': '/path/to/results', 'data_config': {'repo_dir': '/data/cb/mihirb14/projects/DynaProt', 'struc_data_dir': '/data/cb/scratch/datasets/pdb_npz', 'dynam_data_dir': '/data/cb/scratch/datasets/atlas_dynamics_labels', 'max_num_residues': 512}}
aatype: torch.Size([10, 512, 21])
residue_index: torch.Size([10, 512])
all_atom_positions: torch.Size([10, 512, 37, 3])
all_atom_mask: torch.Size([10, 512, 37])
resi_pad_mask: torch.Size([10, 512])
dynamics_means: torch.Size([10, 512, 3])
dynamics_covars: torch.Size([10, 512, 3, 3])
frames: torch.Size([10, 512, 4, 4])


In [4]:
batch_prots["dynamics_covars"].mean()

tensor(0.0047, dtype=torch.float64)

In [5]:
from dynaprot.model.architecture import DynaProt

model = DynaProt(modelconfig)
model

DynaProt(
  (sequence_embedding): Embedding(21, 128)
  (ipa_blocks): ModuleList(
    (0-7): 8 x InvariantPointAttention(
      (linear_q): Linear(in_features=128, out_features=64, bias=True)
      (linear_kv): Linear(in_features=128, out_features=128, bias=True)
      (linear_q_points): Linear(in_features=128, out_features=48, bias=True)
      (linear_kv_points): Linear(in_features=128, out_features=144, bias=True)
      (linear_b): Linear(in_features=128, out_features=4, bias=True)
      (linear_out): Linear(in_features=704, out_features=128, bias=True)
      (softmax): Softmax(dim=-1)
      (softplus): Softplus(beta=1.0, threshold=20.0)
    )
  )
  (mean_predictor): Linear(in_features=128, out_features=3, bias=True)
  (covars_predictor): Linear(in_features=128, out_features=6, bias=True)
  (loss): DynaProtLoss()
)

In [9]:
model.training_step(batch_prots,0)

{'resi_gaussians': {'mse_means': tensor(16.9697, grad_fn=<MseLossBackward0>), 'kldiv': tensor(11190.6953, grad_fn=<MeanBackward0>)}, 'resi_rmsf': {}}
{'resi_gaussians': {'mse_means': 16.969669342041016, 'kldiv': 11190.6953125}, 'resi_rmsf': {}}
tensor(11207.6650, grad_fn=<AddBackward0>)


(tensor(11207.6650, grad_fn=<AddBackward0>),
 {'resi_gaussians': {'mse_means': 16.969669342041016, 'kldiv': 11190.6953125},
  'resi_rmsf': {}})

In [89]:
import torch.nn.functional as F
b = 10
n = 2
preds = torch.repeat_interleave(torch.arange(6)+1,n).reshape(-1,n).T.unsqueeze(0).repeat(b,1,1)
# print(preds,preds.shape)


L = torch.zeros(b,n,3,3)

i = 0
for c in range(3):
    for r in range(c,3):
        L[:,:,r,c] = preds[:,:,i] 
        if r == c:
            L[:,:,r,c] = F.softplus(L[:,:,r,c])
        i+=1

covars = L @ L.transpose(2,3)
print(covars)
# # print(L)
# print(covars)

tensor([[[[ 1.7247,  2.6265,  3.9398],
          [ 2.6265, 20.1455, 26.0907],
          [ 3.9398, 26.0907, 70.0297]],

         [[ 1.7247,  2.6265,  3.9398],
          [ 2.6265, 20.1455, 26.0907],
          [ 3.9398, 26.0907, 70.0297]]],


        [[[ 1.7247,  2.6265,  3.9398],
          [ 2.6265, 20.1455, 26.0907],
          [ 3.9398, 26.0907, 70.0297]],

         [[ 1.7247,  2.6265,  3.9398],
          [ 2.6265, 20.1455, 26.0907],
          [ 3.9398, 26.0907, 70.0297]]],


        [[[ 1.7247,  2.6265,  3.9398],
          [ 2.6265, 20.1455, 26.0907],
          [ 3.9398, 26.0907, 70.0297]],

         [[ 1.7247,  2.6265,  3.9398],
          [ 2.6265, 20.1455, 26.0907],
          [ 3.9398, 26.0907, 70.0297]]],


        [[[ 1.7247,  2.6265,  3.9398],
          [ 2.6265, 20.1455, 26.0907],
          [ 3.9398, 26.0907, 70.0297]],

         [[ 1.7247,  2.6265,  3.9398],
          [ 2.6265, 20.1455, 26.0907],
          [ 3.9398, 26.0907, 70.0297]]],


        [[[ 1.7247,  2.6265,  3.9398],
 